In [29]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *


StatementMeta(, b9be0520-b150-443b-8ab6-9ed34ed2ade5, 31, Finished, Available)

In [30]:
%%sql
CREATE TABLE if not EXISTS lh_ecormece.Gold_DimCustomer
(
    Customer_ID	string	,
    Customer_Name	string	,
    Segment	string	,
    City	string	,
    State	string	,
    Country	string	,
    Region	string	,
    Created_TS	timestamp	,
    Modified_TS	timestamp	  
)
using DELTA



StatementMeta(, b9be0520-b150-443b-8ab6-9ed34ed2ade5, 32, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [32]:
max_date=spark.sql("select coalesce(max(Modified_TS),'1900-01-01') from lh_ecormece.gold_dimcustomer").first()[0]

StatementMeta(, b9be0520-b150-443b-8ab6-9ed34ed2ade5, 34, Finished, Available)

In [33]:
df_bronze = spark.sql("SELECT Customer_ID,Customer_Name,Segment,City,State,Country,Region from lh_ecormece.bronze_sales WHERE Modified_TS >'{}'".format(max_date)).drop_duplicates()

StatementMeta(, b9be0520-b150-443b-8ab6-9ed34ed2ade5, 35, Finished, Available)

In [34]:
df_bronze.createOrReplaceTempView("ViewCustomer")

StatementMeta(, b9be0520-b150-443b-8ab6-9ed34ed2ade5, 36, Finished, Available)

In [35]:
%%sql
MERGE INTO lh_ecormece.Gold_DimCustomer as cm 
using ViewCustomer as vc 
on cm.Customer_ID = vc.Customer_ID
when MATCHED THEN
update set
    cm.Customer_Name	=	vc.Customer_Name,
    cm.Segment	=	vc.Segment	,
    cm.City	=	vc.City	,
    cm.State	=	vc.State	,
    cm.Country	=	vc.Country	,
    cm.Region	=	vc.Region	,
    cm.Modified_TS	=	current_timestamp()	
when not matched then insert
(
cm.Customer_ID	,
cm.Customer_Name	,
cm.Segment	,
cm.City	,
cm.State	,
cm.Country	,
cm.Region	,
cm.Created_TS	,
cm.Modified_TS	
)
values
(
vc.Customer_ID	,
vc.Customer_Name	,
vc.Segment	,
vc.City	,
vc.State	,
vc.Country	,
vc.Region	,
current_timestamp(),
current_timestamp
)

StatementMeta(, b9be0520-b150-443b-8ab6-9ed34ed2ade5, 37, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>